In [ ]:
import os

os.environ["CONDA_PREFIX"] = "/opt/conda"
!pip install -U uv
!uv pip install modAL-python torchvision torchmetrics mlxtend torchsummary


In [4]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
import torchvision
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
from modAL.models import ActiveLearner
from torch.utils.data import DataLoader
import mlxtend
import random
from tqdm.auto import tqdm
from pathlib import Path
import requests
from torchsummary import summary
import torchvision.models as models
from timeit import default_timer as timer
from torchmetrics import F1Score, Precision, Recall
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix
from modAL.uncertainty import uncertainty_sampling, margin_sampling, entropy_sampling
from skorch import NeuralNetClassifier

print(f"mlxtend version: {mlxtend.__version__}")
print("torch", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("torchvision", torchvision.__version__)

mlxtend version: 0.23.1
torch 2.3.0+cpu
CUDA available: False
torchvision 0.18.0+cpu


# We will use the Cifar 101 data set as our first dataset


In [ ]:
cifar_train_data = datasets.CIFAR10(
    root="data", train=True, download=True, transform=ToTensor(), target_transform=None
)

cifar_test_data = datasets.CIFAR10(
    root="data", train=False, download=True, transform=ToTensor(), target_transform=None
)

In [ ]:
len(cifar_train_data), len(cifar_test_data)

In [ ]:
cifar_class_names = cifar_train_data.classes
cifar_class_names

In [ ]:
class_to_idx_cif = cifar_train_data.class_to_idx
class_to_idx_cif

In [ ]:
image, label = cifar_train_data[0]
image, label

In [ ]:
torch.tensor(cifar_train_data.targets)

In [ ]:
torch.tensor(cifar_train_data.targets).shape

In [ ]:
print(
    f"Image Shape: {image.shape} -> [color channels, height, width]"
)  # Our images are gray_scale!
print(f"Label: {cifar_class_names[label]}")

In [ ]:
image = image.permute(1, 2, 0).numpy()
plt.imshow(image, interpolation="bilinear")
plt.title(cifar_class_names[label])
plt.axis(False)

In [ ]:
# Show more images
plt.figure(figsize=(9, 9))
rows, cols = 4, 4

for i in range(1, rows * cols + 1):
    random_index = torch.randint(1, len(cifar_train_data), size=[1]).item()

    plt.subplot(rows, cols, i)

    image, label = cifar_train_data[random_index]

    image = image.permute(1, 2, 0).numpy()

    plt.imshow(image, interpolation="bilinear")
    plt.title(cifar_class_names[label])
    plt.axis(False)

plt.show()

# Unlabeling The Dataset


In [ ]:
# Define the percentage of data without labels
percentage_without_labels = 0.8

# Calculate the number of samples without labels
num_samples_without_labels = int(len(cifar_train_data) * percentage_without_labels)

# Create indices for data with labels and without labels
indices_with_labels = list(range(len(cifar_train_data)))
indices_without_labels = np.random.choice(
    indices_with_labels, num_samples_without_labels, replace=False
)

# Create a new split for unlabeled data
cifar_train_data_unlabeled = torch.utils.data.Subset(
    cifar_train_data, indices_without_labels
)

# Store labels of the unlabeled data
labels_of_unlabeled = torch.tensor(
    [cifar_train_data.targets[idx] for idx in indices_without_labels]
)

# Remove the selected samples from cifar_train_data
cifar_train_data = torch.utils.data.Subset(
    cifar_train_data,
    [idx for idx in indices_with_labels if idx not in indices_without_labels],
)

# Remove the labels from the unlabeled data
for idx in indices_without_labels:
    cifar_train_data_unlabeled.dataset.targets[idx] = -1

In [ ]:
len(cifar_train_data)

In [ ]:
len(cifar_train_data_unlabeled)

In [ ]:
len(labels_of_unlabeled)

In [ ]:
cifar_train_data_unlabeled[0]

In [ ]:
cifar_train_data[1][1]

# Creating the DataLoaders


In [ ]:
BATCH_SIZE_CIF = 32

cifar_train_data_loader = DataLoader(cifar_train_data, BATCH_SIZE_CIF, shuffle=True)
cifar_test_data_loader = DataLoader(cifar_test_data, BATCH_SIZE_CIF, shuffle=False)

In [ ]:
cifar_train_data_loader, cifar_test_data_loader

In [ ]:
print(
    f"Length of Training Data loader: {len(cifar_train_data_loader)}, Batches of {cifar_train_data_loader.batch_size}"
)
print(
    f"Length of Testing Data loader: {len(cifar_test_data_loader)}, Batches of {cifar_test_data_loader.batch_size}"
)

In [ ]:
train_features_batch_cif, train_labels_batch_cif = next(iter(cifar_train_data_loader))
(
    train_features_batch_cif.shape,
    train_labels_batch_cif.shape,
)  # [Batch_Size, Color_Channels, Height, Width] Color Channels First!

In [ ]:
# Visualizing Images in the batch

random_idx = torch.randint(0, len(train_features_batch_cif), size=[1]).item()

img, label = train_features_batch_cif[random_idx], train_labels_batch_cif[random_idx]
img = img.permute(1, 2, 0).numpy()
plt.imshow(img, interpolation="bilinear")
plt.title(cifar_class_names[label])
plt.axis(False)

# Importing and Using ResNet 50 Architecture

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

NameError: name 'torch' is not defined

In [ ]:
model = models.resnet50(pretrained=True).to(device)
model.fc = nn.Linear(model.fc.in_features, 10)
next(model.parameters()).device

In [ ]:
f = nn.Flatten(start_dim=0)  # The default start dim is 1
x = f(torch.randn(10, 7, 7))

x.size()

In [ ]:
rand_image_tensor = torch.randn(size=(32, 3, 32, 32)).to(device)
model(rand_image_tensor)

# Printing the Architecture and Number of Trianing Parameters in each Layer


In [ ]:
summary(model, (3, 32, 32))

# External Helper Functions


In [ ]:
if Path("HelperFunctions.py").is_file():
    print("Helper Functions already exists, skipping downloading")
else:
    print("downloading HelperFunctions.py")

    request = requests.get(
        "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py"
    )

    with open("HelperFunctions.py", "wb") as f:
        f.write(request.content)

In [ ]:
# Picking a loss function and an optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [ ]:
def accuracy_fn(y_pred, y_true):
    correct = torch.eq(y_pred, y_true).sum().item()
    accuracy = correct / len(y_true) * 100
    return accuracy

In [ ]:
def display_train_time(start: float, end: float, device: torch.device = None):
    total_time = end - start
    print(f"Train time on device {device}: {total_time:.3f} seconds")
    return total_time

In [ ]:
start = timer()
end = timer()

display_train_time(start, end, device="cpu")

In [ ]:
def train_step(
    model: nn.Module,
    data_loader: torch.utils.data.dataloader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_fn,
    device: torch.device = device,
):
    training_loss = 0
    training_acc = 0

    model.train()

    for batch, (x, y) in enumerate(data_loader):
        # Put the data on the target device
        x, y = x.to(device), y.to(device)

        # Forward Pass
        y_pred = model(x)

        # Loss
        loss = loss_fn(y_pred, y)
        training_loss += loss
        training_acc += accuracy_fn(y_pred.argmax(dim=1), y)

        # Optimizer zero grad
        optimizer.zero_grad()

        # Loss Backward
        loss.backward()

        # optimizer step step step
        optimizer.step()

    # After looping over batches, devide the training loss over the number of batches to get the averge loss per batch
    training_loss /= len(
        data_loader
    )  # train.data_loader (returns the number of batches)
    training_acc /= len(data_loader)

    print(
        f"Training Loss: {training_loss:.5f} | Training Accuracy: {training_acc:.2f}%"
    )

In [ ]:
def test_step(
    model: nn.Module,
    data_loader: torch.utils.data.dataloader,
    loss_fn: nn.Module,
    accuracy_fn,
    device: torch.device = device,
):
    testing_acc = 0
    testing_loss = 0

    model.eval()
    with torch.inference_mode():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)

            # Forward Pass
            y_test_pred = model(x)

            # Loss
            testing_loss += loss_fn(y_test_pred, y)

            # Accuracy
            testing_acc += accuracy_fn(y_test_pred.argmax(dim=1), y)

        testing_loss /= len(data_loader)
        testing_acc /= len(data_loader)

    print(f"Testing Loss: {testing_loss:.3f} | Testing Accuracy: {testing_acc:.2f}%")

In [ ]:
!nvidia-smi

In [ ]:
# Set the seed and start the timer
torch.manual_seed(42)
torch.cuda.manual_seed(42)

train_start_time_on_gpu = timer()

# Set the number of epochs
epochs = 5

# Training
for epoch in tqdm(range(epochs)):
    print(f"epoch: {epoch}.\n--------------------------------------")

    train_step(model, cifar_train_data_loader, loss_fn, optimizer, accuracy_fn, device)

    # Testing
    test_step(model, cifar_test_data_loader, loss_fn, accuracy_fn, device)

# Compute the time of the training
train_end_time_on_gpu = timer()

display_train_time(
    train_start_time_on_gpu,
    train_end_time_on_gpu,
    device=next(model.parameters()).device,
)

In [ ]:
torch.manual_seed(42)

In [ ]:
def eval_model(
    model: nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    accuracy_fn,
    device: torch.device = device,
):
    model.eval()

    loss, acc = 0, 0

    with torch.inference_mode():
        for x, y in tqdm(data_loader):
            x, y = x.to(device), y.to(device)

            # Forward Pass
            y_pred = model(x)

            # Loss
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(torch.argmax(y_pred, dim=1), y)

        loss /= len(data_loader)
        acc /= len(data_loader)

    return {
        "Model Name": model.__class__.__name__,
        "Model Loss": loss.item(),
        "Model Accuracy": acc,
    }, torch.argmax(y_pred, dim=1)

In [ ]:
# Testing the model
cifar_model_results, y_pred = eval_model(
    model, cifar_test_data_loader, loss_fn, accuracy_fn
)

In [ ]:
cifar_model_results, y_pred

In [ ]:
len(next(iter(cifar_test_data_loader)))

# Evaluating Model Visually


In [ ]:
test_samples = []
test_labels = []

for sample, label in random.sample(list(cifar_test_data), k=9):
    test_samples.append(sample)
    test_labels.append(label)

rows = 6
cols = 5

plt.figure(figsize=(13, 11))

model.eval()
with torch.inference_mode():
    for i in range(1, rows * cols + 1):
        plt.subplot(rows, cols, i)

        random_index = torch.randint(1, 9, size=[1]).item()
        image, label = test_samples[random_index], test_labels[random_index]

        pred_label = model(image.unsqueeze(dim=0).to(device)).argmax()

        image = image.permute(1, 2, 0).numpy()

        plt.imshow(image, interpolation="bilinear")

        if pred_label == label:
            plt.title(
                f"True: {cifar_class_names[label]} | Pred: {cifar_class_names[pred_label]}",
                c="g",
                fontsize=10,
            )
        else:
            plt.title(
                f"True: {cifar_class_names[label]} | Pred: {cifar_class_names[pred_label]}",
                c="r",
                fontsize=10,
            )

        plt.axis(False)

In [ ]:
mlxtend.__version__.split(".")[1]

In [ ]:
type(torch.tensor(cifar_test_data.targets)), type(y_pred)

In [ ]:
y_preds = []
model.eval()

with torch.inference_mode():
    for x, y in tqdm(cifar_test_data_loader, desc="Making Predictions..."):
        # Send the data to the target device
        x, y = x.to(device), y.to(device)

        # Forward Pass
        logits = model(x)

        # Pred probs then labels
        y_pred = torch.softmax(logits, dim=1).argmax(dim=1)

        # Put predictions on the cpu for evaluation
        y_preds.append(y_pred.cpu())

# Concatenate the predicions of all batches
y_pred_tensor = torch.cat(y_preds)

In [ ]:
# Initialize precision, recall, and F1 score metrics
precision = Precision(task="multiclass", num_classes=len(cifar_class_names))
recall = Recall(task="multiclass", num_classes=len(cifar_class_names))
f1 = F1Score(task="multiclass", num_classes=len(cifar_class_names))

# Update the metrics with true and predicted labels
precision.update(y_pred_tensor, torch.tensor(cifar_test_data.targets))
recall.update(y_pred_tensor, torch.tensor(cifar_test_data.targets))
f1.update(y_pred_tensor, torch.tensor(cifar_test_data.targets))

# Compute the metrics
precision_result = precision.compute()
recall_result = recall.compute()
f1_result = f1.compute()

print(f"Precision: {precision_result}")
print(f"Recall: {recall_result}")
print(f"F1 Score: {f1_result}")

In [ ]:
len(y_pred_tensor)

In [ ]:
confmat = ConfusionMatrix(task="multiclass", num_classes=len(cifar_class_names))
confmat_tensor = confmat(
    preds=y_pred_tensor, target=torch.tensor(cifar_test_data.targets)
)
confmat_tensor

In [ ]:
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(),  # Matplotlib likes working with numpy!
    class_names=cifar_class_names,
    figsize=(10, 7),
)

# Saving the Model


In [ ]:
# Create the models path
MODELS_PATH = Path("models")
MODELS_PATH.mkdir(parents=True,
                  exist_ok=True)

# Create model save
MODEL_NAME = "cifer.pth"
MODEL_SAVE_PATH = MODELS_PATH / MODEL_NAME

# Save the model
print(f"Saving model to {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(),
           f=MODEL_SAVE_PATH)

# Load the Model


In [80]:
# Create a model instance
trained_model = models.resnet50().to(device)
trained_model.fc = nn.Linear(model.fc.in_features, 10)
trained_model.load_state_dict(torch.load(MODEL_SAVE_PATH))
trained_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [81]:
summary(trained_model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13            [-1, 256, 8, 8]          16,384
      BatchNorm2d-14            [-1, 25

In [82]:
trained_model = NeuralNetClassifier(
    trained_model,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    verbose=1,
    device=device,
)

# Active Learning

In [83]:
def active_learning(method):
    learner = None
    cycles = 10
    if method == "uncertainty_sampling":
        learner = ActiveLearner(
            estimator=trained_model,
            X_training=cifar_train_data,
            y_training=cifar_train_data.dataset.targets,
            query_strategy=uncertainty_sampling,
        )
    elif method == "margin_sampling":
        learner = ActiveLearner(
            estimator=trained_model,
            X_training=cifar_train_data,
            y_training=cifar_train_data.dataset.targets,
            query_strategy=margin_sampling,
        )
    elif method == "entropy_sampling":
        learner = ActiveLearner(
            estimator=trained_model,
            X_training=cifar_train_data,
            y_training=cifar_train_data.dataset.targets,
            query_strategy=entropy_sampling,
        )
    for cycle in range(cycles):
        print(f"Cycle: {cycle}")
        query_idx, query_instance = learner.query(cifar_train_data_unlabeled)
        learner.teach(
            X=cifar_train_data_unlabeled[query_idx][0],
            y=cifar_train_data_unlabeled[query_idx][1],
        )
        cifar_train_data_unlabeled = np.delete(
            cifar_train_data_unlabeled, query_idx, axis=0
        )
        print(f"Queried Instance: {query_instance}")
        print(f"Queried Index: {query_idx}")
        print(f"Queried Label: {cifar_class_names[query_instance]}")
        print("Queried Image:")
        plt.imshow(cifar_train_data_unlabeled[query_idx][0].permute(1, 2, 0))
        plt.title(cifar_class_names[query_instance])
        plt.axis(False)
        plt.show()
        print("\n\n\n")

In [84]:
active_learning("uncertainty_sampling")

  epoch    train_loss       dur
-------  ------------  --------
      1        2.0030  164.1545
      2        1.7791  210.9525
Cycle: 0
